In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader

In [3]:
import json

In [4]:
with open("bidder-data/data152.json") as f:
    ex = json.load(f)


In [5]:
ex[:3]

[{'rules': ['DefaultPass',
   'OneLevelSuitOpening',
   'OneLevelTakeoutDouble',
   'DefaultPass',
   'SuitResponseToTakeoutDouble',
   'UnforcedRebidOriginalSuitByOpener',
   'LawOfTotalTricks',
   'DefaultPass',
   'LawOfTotalTricks',
   'DefaultPass',
   'DefaultPass',
   'DefaultPass'],
  'board': '4-ba4b7058ecc802db04967b7b5d',
  'calls': ['P', '1D', 'X', 'P', '1S', '2D', '2S', 'P', '4S', 'P', 'P', 'P']},
 {'rules': ['DefaultPass',
   'OneLevelSuitOpening',
   'DefaultPass',
   'OneLevelNewSuitResponse',
   'DefaultPass',
   'RebidOneNotrumpByOpener',
   'DefaultPass',
   'NaturalNotrump',
   'DefaultPass',
   'DefaultPass',
   'DefaultPass'],
  'board': '12-5ca2c074df721394da2eb47e29',
  'calls': ['P', '1C', 'P', '1S', 'P', '1N', 'P', '2N', 'P', 'P', 'P']},
 {'rules': ['OneLevelSuitOpening',
   'DefaultPass',
   'OneLevelNewSuitResponse',
   'DefaultPass',
   'NewSuitByOpener',
   'DefaultPass',
   'ThreeLevelSuitRebidByResponder',
   'DefaultPass',
   'NaturalNotrump',
   'Defau

In [6]:
from game_engine import *

In [7]:
get_board_from_identifier(ex[0]["board"])

{'dealer': 'W',
 'vulnerability': 'Both',
 'hands': {'N': ['7C',
   'QC',
   'KC',
   '4D',
   '8D',
   'TD',
   'QD',
   'KD',
   'AD',
   '2H',
   '8H',
   '9H',
   'JH'],
  'E': ['6C',
   'TC',
   'AC',
   '2D',
   '5H',
   'TH',
   'KH',
   'AH',
   '3S',
   '7S',
   'JS',
   'QS',
   'AS'],
  'S': ['2C',
   '4C',
   '5C',
   '8C',
   '3D',
   '6D',
   'JD',
   '3H',
   '6H',
   'QH',
   '2S',
   '5S',
   '9S'],
  'W': ['3C',
   '9C',
   'JC',
   '5D',
   '7D',
   '9D',
   '4H',
   '7H',
   '4S',
   '6S',
   '8S',
   'TS',
   'KS']}}

In [8]:
ex[0]["calls"]

['P', '1D', 'X', 'P', '1S', '2D', '2S', 'P', '4S', 'P', 'P', 'P']

In [9]:
ex[0]["board"]

'4-ba4b7058ecc802db04967b7b5d'